# Deep Learning - Conditional GAN in PyTorch
---

## Author : Amir Atapour-Abarghouei, amir.atapour-abarghouei@durham.ac.uk

This notebook will provide an example that shows the implementation of a Conditional Generative Adversarial Network in PyTorch.

Copyright (c) 2022 Amir Atapour-Abarghouei, UK.

License : LGPL - http://www.gnu.org/licenses/lgpl.html

We are going to implement a Conditional GAN. Let's start by importing what we need:

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

We should now line up the dataset we are going to use. We will be working with the MNIST dataset.

To process the data, we are going to convert the images to 32x32 images to make the work easier for demonstration purposes.

Since our dataset has ten classes with labels, we are going to use the labels as the auxiliary data for our conditional GAN.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('data', train=True, download=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.Resize([32,32]),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.5], std=[0.5])
    ])),
shuffle=True, batch_size=64, drop_last=True)

print(f"Dataloader created!")

Our dataset is tiny and is not really suited for any real applications, but it will demonstrate the process. Let's look at a few of our images:

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_loader.dataset[i][0].clamp(0,1).repeat(3,1,1).permute(0,2,1).contiguous().permute(2,1,0), cmap=plt.cm.binary)

We can now create our models, we need a Generator and a Discriminator:

In [ ]:
# A few parameters:
n_channels = 1
img_width = 32

# define the generator
class Generator(nn.Module):
    def __init__(self, latent_size=100, label_size=10):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(latent_size+label_size, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, n_channels*img_width*img_width),
            nn.Tanh()
        )
        
    def forward(self, x, c):
        x, c = x.view(x.size(0), -1), c.view(c.size(0), -1).float()
        x = torch.cat((x, c), 1) # [input, label] concatenated
        x = self.layer(x)
        return x.view(x.size(0), n_channels, img_width, img_width)

# define the discriminator
class Discriminator(nn.Module):
    def __init__(self, label_size=10):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(n_channels*img_width*img_width+label_size, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x, c):        
        x, c = x.view(x.size(0), -1), c.view(c.size(0), -1).float()
        x = torch.cat((x, c), 1) # [input, label] concatenated
        return self.layer(x)
        
G = Generator().to(device)
D = Discriminator().to(device)

print(f'Generator has {len(torch.nn.utils.parameters_to_vector(G.parameters()))} parameters.')
print(f'Discriminator has {len(torch.nn.utils.parameters_to_vector(D.parameters()))} parameters')

# initialise the optimiser
optimiser_G = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimiser_D = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
print('Optimisers have been created!')

criterion = nn.BCELoss()
print('Loss function is Binary Cross Entropy!')

Let's start the main training loop now:

In [ ]:
epoch = 0
# training loop
while (epoch<20000):
    
    # arrays for metrics
    logs = {}
    gen_loss_arr = np.zeros(0)
    dis_loss_arr = np.zeros(0)

    # iterate over the train dateset
    for i, batch in enumerate(train_loader):

        x, t = batch
        x, t = x.to(device), t.to(device)

        # convert target labels "t" to a one-hot vector, e.g. 3 becomes [0,0,0,1,0,0,0,...]
        y = torch.zeros(x.size(0), 10).long().to(device).scatter(1, t.view(x.size(0),1), 1)

        # train discriminator
        z = torch.randn(x.size(0), 100).to(device)
        l_r = criterion(D(x,y),       torch.ones([64,1]).to(device)) # real -> 1
        l_f = criterion(D(G(z,y),y), torch.zeros([64,1]).to(device)) # fake -> 0
        loss_d = (l_r + l_f)/2.0
        optimiser_D.zero_grad()
        loss_d.backward()
        optimiser_D.step()
        
        # train generator
        z = torch.randn(x.size(0), 100).to(device)
        loss_g = criterion(D(G(z,y),y), torch.ones([64,1]).to(device)) # fake -> 1
        optimiser_G.zero_grad()
        loss_g.backward()
        optimiser_G.step()

        gen_loss_arr = np.append(gen_loss_arr, loss_g.item())
        dis_loss_arr = np.append(dis_loss_arr, loss_d.item())


    # conditional sample of 10x10
    G.eval()
    print('loss d: {:.3f}, loss g: {:.3f}'.format(gen_loss_arr.mean(), dis_loss_arr.mean()))
    grid = np.zeros([img_width*10, img_width*10])
    for j in range(10):
        c = torch.zeros([10, 10]).to(device)
        c[:, j] = 1
        z = torch.randn(10, 100).to(device)
        y_hat = G(z,c).view(10, img_width, img_width)
        result = y_hat.cpu().data.numpy()
        grid[j*img_width:(j+1)*img_width] = np.concatenate([x for x in result], axis=-1)
    plt.grid(False)
    plt.imshow(grid, cmap='gray')
    plt.show()
    plt.pause(0.0001)
    G.train()

    epoch = epoch+1